In [13]:
import requests
from pathlib import Path
import os 
import sys
import torch
from torch import nn
import pandas as pd
import sklearn
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt

In [ ]:
if Path("../helper_functions.py").is_file():
    print("Helper functions file already exists, skipping download...")
else:
    print("Dowloading helper_functions.py")
    request = requests.get(
        "https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/refs/heads/main/helper_functions.py"
    )
    with open("../src/helper_functions.py", "wb") as f:

        f.write(request.content)

Dowloading helper_functions.py


In [14]:
from timeit import default_timer as Timer
def print_train_time(start: float,
                     end: float,
                     device: torch.device = None):
    """Prints difference between start and end time.
       Use code:
                start_time = timer()
                some code...
                end_time = timer()
                print_train_time(start_time, end_time, device)
    """
    
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time
